# ✂️ Audio Slicer (RMS-based)

This notebook allows you to upload an audio file and automatically slice it into separate files based on silence/activity detection (RMS threshold).

Useful for:
- Extracting individual words from a speech recording.
- Isolating bird calls from a long field recording.
- Splitting musical phrases.

### How it works
It uses `librosa.effects.split`, which calculates the RMS amplitude of the signal and splits it where the signal is above a certain threshold (in decibels relative to the peak).

In [ ]:
# Environment Setup
import sys
import subprocess

try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running on Google Colab")
    
    # Install dependencies
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gradio", "librosa", "soundfile", "matplotlib"], check=True)
    print("✅ Dependencies installed.")
except ImportError:
    IN_COLAB = False
    print("💻 Running locally")

In [ ]:
import gradio as gr
import librosa
import soundfile as sf
import numpy as np
import os
import shutil

print("✅ Libraries imported.")

In [ ]:
def slice_audio(audio_path, threshold_db, min_duration, padding):
    """Slices audio based on silence threshold."""
    if audio_path is None:
        return None, "Please upload an audio file."

    # Create a directory for slices
    output_dir = "slices"
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    try:
        # Load audio
        # sr=None preserves original sampling rate
        y, sr = librosa.load(audio_path, sr=None)
        
        # Split audio
        # top_db: The threshold (in decibels) below reference to consider as silence
        intervals = librosa.effects.split(y, top_db=threshold_db)
        
        saved_files = []
        count = 0
        
        for start, end in intervals:
            # Add padding (in samples)
            pad_samples = int(padding * sr)
            start = max(0, start - pad_samples)
            end = min(len(y), end + pad_samples)
            
            chunk = y[start:end]
            
            # Check minimum duration
            if (len(chunk) / sr) < min_duration:
                continue
                
            # Save slice
            out_name = os.path.join(output_dir, f"slice_{count:03d}.wav")
            sf.write(out_name, chunk, sr)
            saved_files.append(out_name)
            count += 1
            
        if not saved_files:
            return None, "No slices found. Try lowering the threshold or minimum duration."
            
        return saved_files, f"✅ Created {len(saved_files)} slices in '{output_dir}' folder."
        
    except Exception as e:
        return None, f"❌ Error: {str(e)}"

In [ ]:
# Build the Gradio App
with gr.Blocks(title="Audio Slicer") as demo:
    gr.Markdown("# ✂️ Audio Slicer")
    gr.Markdown("Upload audio, set the silence threshold, and download the resulting clips.")
    
    with gr.Row():
        with gr.Column():
            audio_input = gr.Audio(label="Upload Audio", type="filepath")
            
            with gr.Accordion("Settings", open=True):
                threshold_slider = gr.Slider(
                    minimum=10, maximum=80, value=30, step=1, 
                    label="Silence Threshold (dB)", 
                    info="Higher values = more aggressive splitting (treats louder sounds as silence)."
                )
                min_dur_slider = gr.Slider(
                    minimum=0.1, maximum=5.0, value=0.5, step=0.1, 
                    label="Minimum Slice Duration (seconds)"
                )
                padding_slider = gr.Slider(
                    minimum=0.0, maximum=1.0, value=0.1, step=0.05, 
                    label="Padding (seconds)",
                    info="Adds silence/context before and after the slice."
                )
            
            slice_btn = gr.Button("✂️ Slice Audio", variant="primary")
            status_output = gr.Textbox(label="Status", interactive=False)
        
        with gr.Column():
            files_output = gr.File(label="Download Slices", file_count="multiple")

    slice_btn.click(
        fn=slice_audio,
        inputs=[audio_input, threshold_slider, min_dur_slider, padding_slider],
        outputs=[files_output, status_output]
    )

# Handle case where setup cell wasn't run (IN_COLAB undefined)
try:
    share_mode = IN_COLAB
except NameError:
    share_mode = False

demo.launch(share=share_mode)